In [1]:
!pip install ctransformers pypdf langchain torch accelerate bitsandbytes transformers sentence_transformers


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install faiss_cpu streamlit ctransformers streamlit_chat


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [3]:
from langchain.llms import CTransformers

In [4]:
llm = CTransformers(
                model="TheBloke/Llama-2-7B-Chat-GGML",
                model_type="llama",
                max_new_tokens = 512,
                temperature = 0.9
            )

/Users/anu/Downloads/amz-ecom-recommender-main/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 1 files: 100%|██████████| 1/1 [00:00<00:00, 2688.66it/s]


In [5]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import CTransformers
from langchain import PromptTemplate
import langchain
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [6]:
import os

def read_doc(directory):
    file_loader=PyPDFDirectoryLoader(directory)
    documents=file_loader.load()
    return documents




In [7]:
docs = read_doc('covid/')
print(docs)
print(len(docs))

[Document(page_content=" \n   Detail Question and Answers on COVID-19 for Public    What is corona virus  Corona viruses are a large family of viruses which may cause illness in animals or humans. In humans, several coronaviruses are known to cause respiratory infections ranging from the common cold to more severe diseases such as Middle East Respiratory Syndrome (MERS) and Severe Acute Respiratory Syndrome (SARS). The most recently discovered coronavirus causes coronavirus disease COVID-19. What is COVID-19  COVID-19 is the infectious disease caused by the most recently discovered corona virus. This new virus and disease were unknown before the outbreak began in Wuhan, China, in December 2019. What are the symptoms of COVID-19  The most common symptoms of COVID-19 are fever, tiredness, and dry cough. Some patients may have aches and pains, nasal congestion, runny nose, sore throat or diarrhea. These symptoms are usually mild and begin gradually. Some people become infected but don’t d

In [8]:
def chunk_data(docs,chunk_size=500,chunk_overlap=200):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=chunk_size,chunk_overlap=chunk_overlap)
    doc=text_splitter.split_documents(docs)
    return doc

In [9]:
chunks=chunk_data(docs=docs)
len(chunks)

23

In [10]:
hfembeddings = HuggingFaceEmbeddings(
                            model_name="thenlper/gte-large",
                            model_kwargs={'device': 'cpu'}
                        )
vectorstore = FAISS.from_documents(documents=chunks, embedding=hfembeddings)



In [11]:
prompt_temp = '''
With the information provided try to answer the question.
If you cant answer the question based on the information either say you cant find an answer or unable to find an answer.
This is related to medical domain. So try to understand in depth about the context and answer only based on the information provided. Dont generate irrelevant answers

Context: {context}
Question: {question}
Do provide only correct answers

Correct answer:
    '''


In [12]:
custom_prompt_temp = PromptTemplate(template=prompt_temp,
                            input_variables=['context', 'question'])

In [13]:
chain = RetrievalQA.from_chain_type(
                                llm=llm,
                                chain_type="stuff",
                                retriever=vectorstore.as_retriever(search_kwargs={'k': 2}),
                                return_source_documents=True,
                                chain_type_kwargs={"prompt": custom_prompt_temp})

In [14]:
chain = chain("What are the symptoms of covid-19 ?")
chain

{'query': 'What are the symptoms of covid-19 ?',
 'result': '1. Fever.\n    2. Tiredness.\n    3. Dry Cough.',
 'source_documents': [Document(page_content="were unknown before the outbreak began in Wuhan, China, in December 2019. What are the symptoms of COVID-19  The most common symptoms of COVID-19 are fever, tiredness, and dry cough. Some patients may have aches and pains, nasal congestion, runny nose, sore throat or diarrhea. These symptoms are usually mild and begin gradually. Some people become infected but don’t develop any symptoms and don't feel unwell. Most  people (about 80%) recover from the disease without needing special treatment.", metadata={'source': 'covid/covid_QA.pdf', 'page': 0}),
  Document(page_content='as Middle East Respiratory Syndrome (MERS) and Severe Acute Respiratory Syndrome (SARS). The most recently discovered coronavirus causes coronavirus disease COVID-19. What is COVID-19  COVID-19 is the infectious disease caused by the most recently discovered coron

In [15]:
chain = RetrievalQA.from_chain_type(
                                llm=llm,
                                chain_type="stuff",
                                retriever=vectorstore.as_retriever(search_kwargs={'k': 2}),
                                return_source_documents=True,
                                chain_type_kwargs={"prompt": custom_prompt_temp})

In [16]:
ch = chain("How to protect myself from covid-19 ?")
ch

{'query': 'How to protect myself from covid-19 ?',
 'result': '1. To avoid infecting others by going out in public without wearing a mask.\n    2. Avoiding contact with others and visiting medical facilities will allow these facilities to operate more effectively and help protect you and others from possible COVID-19 and other viruses.\n    3. If you develop fever, cough, and difficulty breathing, seek medical advice promptly as this may be due to a respiratory infection or other serious condition.',
 'source_documents': [Document(page_content='person or have       diabetes,       heart       or        lung        disease. Why? You have a higher chance of catching COVID-19 in one of these areas.  Protection measures for persons who are in or have recently visited (past 14 days) areas where COVID-19 is spreading  o Follow the guidance outlined above (Protection measures for everyone) o Self-isolate by staying at home if you begin to feel unwell, even with mild symptoms such as headache,